<a href="https://colab.research.google.com/github/tommygrangel/backtrader/blob/master/unusualActivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

author: u/gigantoir

This notebook screens S&P component stocks for unusual volume relative to open
interest for a given expiration date. Currently, the criteria is open interest 
\> 200 and volume/open interest \> .5. This could be tweaked and I'm open to
feedback but would mostly like if you did that coding yourself. Note that 
historical options data is really hard to find and I can't get change in IV 
although that would be cool. 

Code takes a while to run as it's not super efficient, but you should be good
within like 5 minutes. Download that CSV and do some Shkreli no mouse Excel
speedruns on it.

Feel free to DM me with questions / suggestions but please stop asking me to
teach you how to code. ALSO if you find free money you are required to DM me 
about it.

Have fun autists. Mods r gay. Long live WSB REEEEEEEEEEEEEEEEEEEE


In [ ]:
!pip install pandas_datareader
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=6f74a3221e3bf701ac868cda6b77a655619e640d407a4110907b8cbef6ea6578
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [ ]:
#Imports, keys here are pandas_datareader allows us to download easily and
#yfinance allows us to get into yahoo
import pandas_datareader.data as data
import yfinance as yf
import pandas as pd

yf.pdr_override()

In [ ]:
#S&P components list that we'll be looping through. You can add your retarded
#meme stocks here like SPCE if you want
sp_list = ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'COTY', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'ETFC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'EVRG', 'ES', 'RE', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FLIR', 'FLS', 'FMC', 'F', 'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN', 'GS', 'GWW', 'HRB', 'HAL', 'HBI', 'HOG', 'HIG', 'HAS', 'HCA', 'PEAK', 'HP', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HFC', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HPQ', 'HUM', 'HBAN', 'HII', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'IR', 'INTC', 'ICE', 'IBM', 'INCY', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY', 'J', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM', 'JNPR', 'KSU', 'K', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KSS', 'KHC', 'KR', 'LB', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LEG', 'LDOS', 'LEN', 'LLY', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LYB', 'MTB', 'M', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MKC', 'MXIM', 'MCD', 'MCK', 'MDT', 'MRK', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MHK', 'TAP', 'MDLZ', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'MYL', 'NDAQ', 'NOV', 'NTAP', 'NFLX', 'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NBL', 'JWN', 'NSC', 'NTRS', 'NOC', 'NLOK', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'ORLY', 'OXY', 'ODFL', 'OMC', 'OKE', 'ORCL', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PBCT', 'PEP', 'PKI', 'PRGO', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTN', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SLG', 'SNA', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STE', 'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT', 'TEL', 'FTI', 'TFX', 'TXN', 'TXT', 'TMO', 'TIF', 'TJX', 'TSCO', 'TDG', 'TRV', 'TFC', 'TWTR', 'TSN', 'UDR', 'ULTA', 'USB', 'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UTX', 'UHS', 'UNM', 'VFC', 'VLO', 'VAR', 'VTR', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VIAC', 'V', 'VNO', 'VMC', 'WRB', 'WAB', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WDC', 'WU', 'WRK', 'WY', 'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XRX', 'XLNX', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']

In [ ]:
import sys, os
from datetime import date


def unusualActivity(calls_or_puts, exp_date, stocklist):
    i = 1
    old_stdout = sys.stdout
    print(exp_date)
    finaldf = pd.DataFrame()
    for x in stocklist:
        print(x)
        if i%100 == 0:
            print(i)
        i +=1
        sys.stdout = open(os.devnull, "w") 
        ticker = yf.Ticker(x)
        try:
            opt = ticker.option_chain(exp_date)
            if calls_or_puts == 'calls':
                opt.calls.insert(0, 'Exp_Date', exp_date)
                opt.calls.insert(0, 'Symbol', x)
                opt.calls.insert(3, 'stock_price', data.get_data_yahoo(x, end_date = date.today())['Close'][-1])
                opt.calls['V/OI'] = (opt.calls['volume'].astype('float')/opt.calls['openInterest']) 
                finaldf = finaldf.append(opt.calls[opt.calls['volume'] > 200]) #finaldf.append(opt.calls[(opt.calls['volume'].astype('float')/opt.calls['openInterest'] > .5) & (opt.calls['openInterest'] > 200)])
                sys.stdout = old_stdout
            elif calls_or_puts == 'puts':
                opt.puts.insert(0, 'Exp_Date', exp_date)
                opt.puts.insert(0, 'Symbol', x)
                opt.puts.insert(3, 'stock_price', data.get_data_yahoo(x, end_date = date.today())['Close'][-1])
                opt.puts['stock_price'] = data.get_data_yahoo(x, end_date = date.today())['Close'][-1]
                opt.puts['V/OI'] = (opt.puts['volume'].astype('float')/opt.puts['openInterest'])
                finaldf = finaldf.append(opt.puts[opt.puts['volume'] > 200]) #(opt.puts[(opt.puts['volume'].astype('float')/opt.puts['openInterest'] > .5) & (opt.puts['openInterest'] > 200)])
                sys.stdout = old_stdout
            else:
                print('set calls_or_puts equal to calls or puts retard')
                break
        except:
            pass
            sys.stdout = old_stdout
    return finaldf

In [ ]:
calls_or_puts = 'puts'
exp_date_list = ['2020-07-17']#, '2020-04-16', '2020-05-14', '2020-06-18', '2020-07-16']
final = pd.DataFrame()

for exp_date in exp_date_list:
    returned = unusualActivity(calls_or_puts, exp_date, sp_list)
    
    returned = returned.drop(columns = ['contractSymbol', 'lastTradeDate', 'contractSize', 'currency'])
    returned.insert(3, 'Distance OTM', returned['stock_price'] - returned['strike'])
    returned['Value'] = returned['openInterest']*returned['lastPrice']*100
    
    final = final.append(returned)

final

AGN
ADS
LNT
ALL


Exception in thread Thread-45:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/yfinance/multi.py", line 167, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "/usr/local/lib/python3.6/dist-packages/yfinance/multi.py", line 182, in _download_one
    rounding=rounding, many=True)
  File "/usr/local/lib/python3.6/dist-packages/yfinance/base.py", line 155, in history
    data = data.json()
  File "/usr/local/lib/python3.6/dist-packages/requests/models.py", line 898, in json
    return complexjson.loads(self.text, **kwargs)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    ret

GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS


Exception in thread Thread-80:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/yfinance/multi.py", line 167, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "/usr/local/lib/python3.6/dist-packages/yfinance/multi.py", line 182, in _download_one
    rounding=rounding, many=True)
  File "/usr/local/lib/python3.6/dist-packages/yfinance/base.py", line 155, in history
    data = data.json()
  File "/usr/local/lib/python3.6/dist-packages/requests/models.py", line 898, in json
    return complexjson.loads(self.text, **kwargs)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    ret

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MMM200717P00070000,2020-06-09 23:00:26,70.0,0.09,0.00,0.13,0.000000,0.000000,8.0,6.0,1.300785,False,REGULAR,USD
1,MMM200717P00075000,2020-06-09 23:00:26,75.0,0.36,0.00,0.16,0.000000,0.000000,10.0,10.0,1.222660,False,REGULAR,USD
2,MMM200717P00080000,2020-06-09 23:00:26,80.0,0.08,0.00,0.16,0.000000,0.000000,2.0,12.0,1.121098,False,REGULAR,USD
3,MMM200717P00085000,2020-06-19 19:41:01,85.0,0.05,0.00,0.16,0.000000,0.000000,1.0,41.0,1.023442,False,REGULAR,USD
4,MMM200717P00090000,2020-06-11 18:38:39,90.0,0.02,0.00,0.16,0.000000,0.000000,1.0,70.0,0.931641,False,REGULAR,USD
5,MMM200717P00095000,2020-05-29 13:30:06,95.0,0.04,0.00,0.05,0.000000,0.000000,3.0,27.0,0.742190,False,REGULAR,USD
6,MMM200717P00100000,2020-06-17 19:36:19,100.0,0.05,0.00,0.17,0.040000,400.000000,5.0,119.0,0.767580,False,REGULAR,USD
7,MMM200717P00105000,2020-06-18 17:08:10,105.0,0.09,0.00,0.15,0.000000,0.000000,25.0,237.0,0.675784,False,REGULAR,USD
8,MMM200717P00110000,2020-06-26 19:58:09,110.0,0.09,0.06,0.19,0.040000,80.000010,3.0,558.0,0.644535,False,REGULAR,USD
9,MMM200717P00115000,2020-06-22 16:35:51,115.0,0.18,0.01,0.46,0.070000,63.636368,2.0,714.0,0.627933,False,REGULAR,USD


In [ ]:
#Download the CSV and you're done!
from google.colab import files

returned.to_csv('unusual_puts_activity_7.17_exp.csv')
files.download('unusual_puts_activity_7.17_exp.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>